In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import uuid
import shutil
import json
import tempfile
import platform
import subprocess
import statistics
import random as rd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from anacore.msi import MSISample, LocusRes, LocusResPairsCombi, MSILocus, MSIReport, LocusClassifier
from anacore.msings import MSINGSAnalysis, MSINGSReport
sns.set_style("whitegrid")

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

In [ ]:
def mergeSeries(series):
    count_by_val = dict()
    for curr_series in series:
        for val in curr_series:
            if val in count_by_val:
                count_by_val[val] += 1
            else:
                count_by_val[val] = 1
    new_series = list()
    for val, count in count_by_val.items():
        for idx in range(count):
            new_series.append(val)
    new_series = np.asarray(new_series)
    return new_series

def getStacked(array):
    nb_by_val = {}
    for curr_val in array:
        curr_val = str(curr_val)
        if curr_val in nb_by_val:
            nb_by_val[curr_val] += 1
        else:
             nb_by_val[curr_val] = 1
    return nb_by_val

def getUnstacked(nb_by_val):
    array = []
    for val, nb in nb_by_val.items():
        for idx in range(nb):
            array.append(int(val))
    return array

def getMSISamples(in_folder):
    samples_res = list()
    for filename in os.listdir(in_folder):
            filepath = os.path.join(in_folder, filename)
            data = None
            with open(filepath) as FH:
                data = json.load(FH)[0]
            curr_spl = MSISample.fromDict(data)
            samples_res.append(curr_spl)
    return samples_res

def getExpectedStatus(in_status, in_separator="\t"):
    status_by_spl = dict()
    with open(in_status) as FH_status:
        for line in FH_status:
            if not line.startswith("#"):
                spl, date, report_id, commentary, spl_status, BAT25_status, BAT26_status, NR21_status, NR22_status, NR24_status, NR27_status, HT17_status, commentary_2 = [elt.strip() for elt in line.split(in_separator)]
                if "normal" not in commentary and "sain" not in commentary:
                    status_by_spl[spl] = {
                        "MSI_BAT25": BAT25_status,
                        "MSI_BAT26": BAT26_status,
                        "MSI_NR21": NR21_status,
                        "MSI_NR22": NR22_status,
                        "MSI_NR24": NR24_status,
                        "MSI_NR27": NR27_status,
                        "MSI_HT17": HT17_status,
                        "sample": spl_status
                    }
    return status_by_spl

In [ ]:
def cross_val_score_custom(locus, whole_data, whole_labels, classifiers, cv):
    rows = []
    titles = ["locus", "dataset_id", "train_nb_MSS", "train_nb_MSI", "test_nb_MSS", "test_nb_MSI", "%_error_MSS", "%_error_MSI", "method", "accuracy"]
    trainnig_id = 0
    for train_idx, test_idx in cv.split(whole_data, groups=whole_labels):
        trainnig_id += 1
        # Split dataset in train and test sub-datasets
        train = {"names": [], "data": [], "labels": []}
        test = {"names": [], "data": [], "labels": []}
        for data_idx, (data, label) in enumerate(zip(whole_data, whole_labels)):
            lib_name = data.name.replace("_L001", "")
            spl_name = lib_name.split("_")[0]
            if data_idx in train_idx:
                train["names"].append(lib_name)
                train["data"].append(data)
                train["labels"].append(label)
            elif data_idx in test_idx:
                test["names"].append(lib_name)
                test["data"].append(data)
                test["labels"].append(label)
        # Launch classifier
        for clf in classifiers:
            predictions = []
            if clf.method_name == "MSINGS":
                if platform.system() == "Linux":
                    clf.fit(train["data"])
                    predictions = clf.predict(test["data"])
                    clf.clearTmp()
            else:
                clf.fit(train["data"])
                predictions = clf.predict(test["data"])
            if len(predictions) != 0:
                accuracy = getAccuracy(*getDetermined(predictions, test["labels"]))
                err_by_status = getNbErrByStatus(predictions, test["labels"])
                err_MSS = 0 if "MSS" not in err_by_status else err_by_status["MSS"]
                err_MSI = 0 if "MSI" not in err_by_status else err_by_status["MSI"]
                rows.append([
                    locus["name"],
                    trainnig_id,
                    train["labels"].count("MSS"),
                    train["labels"].count("MSI"),
                    test["labels"].count("MSS"),
                    test["labels"].count("MSI"),
                    (100 * err_MSS)/test["labels"].count("MSS"),
                    (100 * err_MSI)/test["labels"].count("MSI"),
                    clf.method_name,
                    accuracy
                ])
                
    return pd.DataFrame.from_records(rows, columns=titles)     


def getDetermined(predictions, labels):
    cleaned_pred = list()
    cleaned_lab = list()
    for observed, expected in zip(predictions, labels):
        if expected != "Undetermined":
            cleaned_pred.append(observed)
            cleaned_lab.append(expected)
    return cleaned_pred, cleaned_lab


def getNbErrByStatus(predictions, labels):
    nb_err = {elt: 0 for elt in set(labels)}
    for expected, observed in zip(labels, predictions):
        for status in nb_err:
            if expected == status:
                if expected != observed:
                    nb_err[status] += 1
    return nb_err


def getAccuracy(predictions, labels):
    nb_true = 0
    nb_false = 0
    for idx, predicted in enumerate(predictions):
        expected = labels[idx]
        if expected == predicted:
            nb_true += 1
        else:
            nb_false += 1
    # Calculate accuracy   
    return nb_true/(nb_true + nb_false)


class MSINGSClassifier:
    def __init__(self, msings_env, msi_path, locus_id):
        self.locus_id = locus_id
        self.method_name = "MSINGS"
        self.model_method_name = "model"
        self.msings_env = msings_env
        self.msi_path = msi_path
        self.id = str(uuid.uuid4())
        self.working_directory = os.path.join(tempfile.gettempdir(), self.id)
        self.train_directory = os.path.join(self.working_directory, "train")
        self.predict_directory = os.path.join(self.working_directory, "predict")
        self.baseline_path = os.path.join(self.train_directory, "model.tsv")
        self.classes_ = ["MSI", "MSS"]
        
    def fit(self, data):
        if not os.path.exists(self.train_directory):
            os.makedirs(self.train_directory)
        # Analyse training samples
        tmp_files = []
        for msi_idx, msi in enumerate(data):
            if msi.loci[self.locus_id].results[self.model_method_name].status == "MSS":  # Only the MSS are required for baseline creation
                model_path = os.path.join(self.train_directory, "spl{}.msi.txt".format(msi_idx))
                tmp_files.append(model_path)
                with MSINGSAnalysis(model_path, "w") as FH_analyze:
                    FH_analyze.write(msi.loci[self.locus_id])
        # Create model
        cmd = [msings_env, msi_path, "create_baseline", self.train_directory, "-o", self.baseline_path]
        subprocess.check_call(cmd)
        # Removes tmp analyses
        for curr_tmp in tmp_files:
            if os.path.exists(curr_tmp):
                os.remove(curr_tmp)
        return self
            
    def predict(self, data):
        classes = []
        idx_by_class = {cls:idx for idx, cls in enumerate(self.classes_)}
        if not os.path.exists(self.predict_directory):
            os.makedirs(self.predict_directory)
        for msi_idx, msi in enumerate(data):
            msi_locus = msi.loci[self.locus_id]
            # Classify
            analyze_path = os.path.join(self.predict_directory, "spl{}.msi.txt".format(msi_idx))
            report_path = os.path.join(self.predict_directory, "spl{}_report.tsv".format(msi_idx))
            with MSINGSAnalysis(analyze_path, "w") as FH_analyze:
                FH_analyze.write(msi_locus)
            cmd = [self.msings_env, self.msi_path, "count_msi_samples", self.baseline_path, self.predict_directory, "-m", "2.0", "-t", "0.2", "0.2", "-o", report_path]
            subprocess.check_call(cmd)
            # mSINGS parse results
            report = MSINGSReport(report_path)
            predicted_status = report.samples["spl{}".format(msi_idx)].loci[msi_locus.position].results["MSINGS"].status
            classes.append(predicted_status)
            # Removes tmp files
            for curr_tmp in [analyze_path, report_path]:
                if os.path.exists(curr_tmp):
                    os.remove(curr_tmp)
        return classes
    
    def __del__(self):
        self.clearTmp()
    
    def clearTmp(self):
        if os.path.exists(self.working_directory):
            shutil.rmtree(self.working_directory)

In [ ]:
# Parameters
min_nb_reads = 300
min_nb_reads_models = 800
init_random_seed = 8852
method_name = "MIAmSClassif"

msings_env = os.path.join(os.getcwd(), "msings", "msings-env", "bin", "python")
msi_path = os.path.join(os.getcwd(), "msings", "msi")

## Locus models from runs

### 1- Models for MSS

In [ ]:
# Load expected status
status_path = os.path.join(os.getcwd(), "data", "status_by_spl.tsv")
status_by_spl = getExpectedStatus(status_path)

# Import results
data_folder = os.path.join(os.getcwd(), "data")
msi_eval = []
runs_parent_folder = os.path.join(data_folder, "MIAmS_1.0.0-b4")
for run_folder in os.listdir(runs_parent_folder):
    filepath = os.path.join(runs_parent_folder, run_folder)
    msi_eval.extend(getMSISamples(os.path.join(filepath, "data")))

# Get loci
loci = set()
loci_id_by_name = {}
for spl in msi_eval:
    for locus_id in spl.loci:
        loci.add(locus_id)
        locus_name = spl.loci[locus_id].name
        loci_id_by_name[locus_name] = locus_id

# Get models params
models_param = list()
missing_spl = set()
for locus_id in sorted(loci):
    means = list()
    std_dev = list()
    locus_name = msi_eval[0].loci[locus_id].name
    for spl in msi_eval:
        diamic_name = spl.name.replace("_L001", "").split("_")[0]
        if diamic_name in status_by_spl:
            if status_by_spl[diamic_name][locus_name] == "MSS" and spl.loci[locus_id].results[method_name].getNbFrag() * 2 >= min_nb_reads_models:
                series = getUnstacked(spl.loci[locus_id].results[method_name].data["nb_by_length"])
                means.append(statistics.mean(series))
                std_dev.append(statistics.stdev(series))
                sns.kdeplot(series, label=spl.name, legend=False, bw=0.5)
        else:
            missing_spl.add(diamic_name)
    if len(means) > 3:
        print("{}: {:0.2f} (+/- {:0.2f}) on {} samples".format(locus_name, statistics.median(means), statistics.median(std_dev), len(means)))
        models_param.append({
            "name": locus_name,
            "id": locus_id,
            "mean": round(statistics.median(means), 0),
            "std_dev": round(statistics.median(std_dev), 2),
            "simulated_data": []
        })
    else:
        print("{}: ? (+/- ?)".format(locus_name))

    plt.title("{} {} samples".format(locus_name, len(means)))
    plt.show()
print("\nSamples not in status database {}".format(sorted(missing_spl)))

Les profils étudiés dans les runs sont très similaires si l'on se concentre sur ceux contenant plus de {{min_nb_reads}} reads.

## Evaluation at locus level

#### 1- Classification accuracy and stability

In [ ]:
# Classify data with ShuffleSplit
accuracy_locus_df = []
for locus in models_param:
    # Classifiers
    used_classifiers = [
        LocusClassifier(
            locus["id"], "Logistic Regression", LogisticRegression(random_state=init_random_seed), data_method_name=method_name
        ),
        LocusClassifier(
            locus["id"], "Decision Tree", DecisionTreeClassifier(random_state=init_random_seed), data_method_name=method_name
        ),
        LocusClassifier(
            locus["id"], "K-Neighbors", KNeighborsClassifier(2), data_method_name=method_name
        ),
        LocusClassifier(
            locus["id"], "Gaussian Naive Bayes", GaussianNB(), data_method_name=method_name
        ),
        LocusClassifier(
            locus["id"], "C-Support Vector", SVC(random_state=init_random_seed, probability=True), data_method_name=method_name
        ),
        LocusClassifier(
            locus["id"], "Random Forest", RandomForestClassifier(random_state=init_random_seed), data_method_name=method_name
        ),
        LocusClassifier(
            locus["id"], "Random Forest 20", RandomForestClassifier(random_state=init_random_seed, n_estimators=20), data_method_name=method_name
        ),
        LocusClassifier(
            locus["id"], "Random Forest 30", RandomForestClassifier(random_state=init_random_seed, n_estimators=30), data_method_name=method_name
        ),
        MSINGSClassifier(msings_env, msi_path, locus["id"])
    ]
    
    # Select data
    whole_labels = []
    whole_data_obj = []
    for spl in msi_eval:
        if spl.loci[locus["id"]].results[method_name].getNbFrag() * 2 >= min_nb_reads:
            diamic_name = spl.name.replace("_L001", "").split("_")[0]
            if diamic_name in status_by_spl:
                status = status_by_spl[diamic_name][locus["name"]]
                if status != "Undetermined":
                    whole_labels.append(status)
                    whole_data_obj.append(spl)
                    spl.loci[locus["id"]].results["model"] = LocusResPairsCombi(status, data=spl.loci[locus["id"]].results[method_name].data)
    print("{}: {} stables, {} instables".format(locus["name"], whole_labels.count("MSS"), whole_labels.count("MSI")))
    
    # Calculates results
    cv = ShuffleSplit(n_splits=200, test_size=0.4, random_state=init_random_seed)
    accuracy_locus_df.append(
        cross_val_score_custom(locus, whole_data_obj, whole_labels, used_classifiers, cv)
    )
accuracy_df = pd.concat(accuracy_locus_df)

In [ ]:
# Plot dataset information
accuracy_plt = sns.boxplot(x="locus", y="train_nb_MSS", hue="method", data=accuracy_df, medianprops=dict(linewidth=2, color='firebrick'))
plt.subplots_adjust(top=0.95)
plt.gcf().suptitle("Number of MSS in train dataset")
plt.show()

accuracy_plt = sns.boxplot(x="locus", y="train_nb_MSI", hue="method", data=accuracy_df, medianprops=dict(linewidth=2, color='firebrick'))
plt.subplots_adjust(top=0.95)
plt.gcf().suptitle("Number of MSI in train dataset")
plt.show()

accuracy_plt = sns.boxplot(x="locus", y="test_nb_MSS", hue="method", data=accuracy_df, medianprops=dict(linewidth=2, color='firebrick'))
plt.subplots_adjust(top=0.95)
plt.gcf().suptitle("Number of MSS in test dataset")
plt.show()

accuracy_plt = sns.boxplot(x="locus", y="test_nb_MSI", hue="method", data=accuracy_df, medianprops=dict(linewidth=2, color='firebrick'))
plt.subplots_adjust(top=0.95)
plt.gcf().suptitle("Number of MSI in test dataset")
plt.show()

* Le nombre d'échantillons stables et instables est très limite dans une partie importante des jeux de test de BAT25. Il faudra en tenir compte dans les conclusions de cette étude en limitant l'importance des résultats obtenus sur ce marqueur.
* Globalement nous avons peu de données pour de l'apprentissage.

##### Accuracy

In [ ]:
# Plot accuracy
accuracy_plt = sns.factorplot(x="locus", y="accuracy", hue="method", data=accuracy_df, kind="bar", size=7)
for ax in accuracy_plt.axes.flat:
    for item in ax.patches:
        height = 0 if np.isnan(item.get_height()) else item.get_height()
        ax.annotate(
            '{:.1%}'.format(item.get_height()),
            (
                item.get_x() + item.get_width() / 2.,
                item.get_height()
            ),
            ha='center', va='center', fontsize=11, color='gray', rotation=90, xytext=(0, 20),
            textcoords='offset points')
    ax.set_ylim(0.5, 1.0)
plt.subplots_adjust(top=0.9)
plt.gcf().suptitle("Classification accuracy")
plt.show()

# Plot zoomed accuracy
accuracy_plt = sns.factorplot(x="locus", y="accuracy", hue="method", data=accuracy_df, kind="bar", size=7)
for ax in accuracy_plt.axes.flat:
    ax.set_ylim(0.95, 1.0)
plt.subplots_adjust(top=0.95)
plt.gcf().suptitle("Zoomed classification accuracy")
plt.show()

# Plot accuracy boxplot
plt.figure(figsize=(8,5))
accuracy_plt = sns.boxplot(x="locus", y="accuracy", hue="method", data=accuracy_df, medianprops=dict(linewidth=2, color='firebrick'))
plt.subplots_adjust(top=0.95)
plt.gcf().suptitle("Classification accuracy")
plt.show()

* Le C-Support vector a la meilleure accuracy pour 5 des 6 marqueurs.
* Pour HT17 c'est la Logistic Regression, le K-Neighbors et le C-Support Vector qui ont les meilleurs résultats.

In [ ]:
"""
# Accuracy summary in dataframe
clf_names = [clf.method_name for clf in used_classifiers]

cols = {"locus": [], "method": [], "accuracy": [], "stdev": []}
for locus in models_param:
    for idx, clf_name in enumerate(clf_names):
        scores = accuracy_df[(accuracy_df.method == clf_name) & (accuracy_df.locus == locus["name"])]["accuracy"]
        cols["locus"].append(locus["name"])
        cols["method"].append(clf_name)
        cols["accuracy"].append(scores.mean())
        cols["stdev"].append(scores.std())
accuracy_summary_df = pd.DataFrame(data=cols)
accuracy_summary_df
"""

##### Errors by status

In [ ]:
# Error type
plt.figure(figsize=(8,5))
error_plt = sns.boxplot(x="locus", y="%_error_MSS", hue="method", data=accuracy_df, medianprops=dict(linewidth=2, color='firebrick'))
plt.subplots_adjust(top=0.95)
error_plt.set_ylim(0, 30)
plt.gcf().suptitle("% of classification error on MSS")
plt.show()

plt.figure(figsize=(8,5))
accuracy_plt = sns.boxplot(x="locus", y="%_error_MSI", hue="method", data=accuracy_df, medianprops=dict(linewidth=2, color='firebrick'))
plt.subplots_adjust(top=0.95)
plt.gcf().suptitle("% of classification error on MSI")
plt.show()

* Les erreurs sont en majorité faites sur des MSI pris pour des MSS.
* Sur les MSS le C-Support Vector est le meilleur ou fait partie des meilleurs classifieurs quelque soit le marqueur.
* Sur les MSI le K-Neighbors et le C-Support Vector ont les meilleurs résultats.

##### Accuracy and size of training set

In [ ]:
# Classify data with ShuffleSplit
train_size_locus_df = []
for locus in models_param:
    # Classifiers
    used_classifiers = [
        LocusClassifier(
            locus["id"], "Logistic Regression", LogisticRegression(random_state=init_random_seed), data_method_name=method_name
        ),
        LocusClassifier(
            locus["id"], "Decision Tree", DecisionTreeClassifier(random_state=init_random_seed), data_method_name=method_name
        ),
        LocusClassifier(
            locus["id"], "K-Neighbors", KNeighborsClassifier(2), data_method_name=method_name
        ),
        LocusClassifier(
            locus["id"], "Gaussian Naive Bayes", GaussianNB(), data_method_name=method_name
        ),
        LocusClassifier(
            locus["id"], "C-Support Vector", SVC(random_state=init_random_seed, probability=True), data_method_name=method_name
        ),
        LocusClassifier(
            locus["id"], "Random Forest", RandomForestClassifier(random_state=init_random_seed), data_method_name=method_name
        ),
        LocusClassifier(
            locus["id"], "Random Forest 20", RandomForestClassifier(random_state=init_random_seed, n_estimators=20), data_method_name=method_name
        ),
        LocusClassifier(
            locus["id"], "Random Forest 30", RandomForestClassifier(random_state=init_random_seed, n_estimators=30), data_method_name=method_name
        ),
        MSINGSClassifier(msings_env, msi_path, locus["id"])
    ]

    # Select data
    whole_labels = []
    whole_data_obj = []
    for spl in msi_eval:
        if spl.loci[locus["id"]].results[method_name].getNbFrag() * 2 >= min_nb_reads:
            diamic_name = spl.name.replace("_L001", "").split("_")[0]
            if diamic_name in status_by_spl:
                status = status_by_spl[diamic_name][locus["name"]]
                if status != "Undetermined":
                    whole_labels.append(status)
                    whole_data_obj.append(spl)

    # Calculates results
    print("Start locus {} ({} MSS, {} MSI)".format(locus["name"], whole_labels.count("MSS"), whole_labels.count("MSI")))
    max_size = int(0.65*len(whole_labels))
    for train_size in range(25, max_size, 5):
        cv = ShuffleSplit(n_splits=200, test_size=(len(whole_labels) - train_size), train_size=train_size, random_state=init_random_seed)
        train_size_locus_df.append(
            cross_val_score_custom(locus, whole_data_obj, whole_labels, used_classifiers, cv)
    )
train_size_df = pd.concat(train_size_locus_df)

In [ ]:
# Add train_nb
def getTrainSize(row):
    return row["train_nb_MSS"] + row["train_nb_MSI"]

train_size_df["train_nb"] = train_size_df.apply(getTrainSize, axis=1)

In [ ]:
# Plot dataset information
accuracy_plt = sns.boxplot(x="locus", y="train_nb_MSS", hue="method", data=train_size_df, medianprops=dict(linewidth=2, color='firebrick'))
plt.subplots_adjust(top=0.95)
plt.gcf().suptitle("Number of MSS in train dataset")
plt.show()

accuracy_plt = sns.boxplot(x="locus", y="train_nb_MSI", hue="method", data=train_size_df, medianprops=dict(linewidth=2, color='firebrick'))
plt.subplots_adjust(top=0.95)
plt.gcf().suptitle("Number of MSI in train dataset")
plt.show()

accuracy_plt = sns.boxplot(x="locus", y="test_nb_MSS", hue="method", data=train_size_df, medianprops=dict(linewidth=2, color='firebrick'))
plt.subplots_adjust(top=0.95)
plt.gcf().suptitle("Number of MSS in test dataset")
plt.show()

accuracy_plt = sns.boxplot(x="locus", y="test_nb_MSI", hue="method", data=train_size_df, medianprops=dict(linewidth=2, color='firebrick'))
plt.subplots_adjust(top=0.95)
plt.gcf().suptitle("Number of MSI in test dataset")
plt.show()

In [ ]:
# Accuracy summary in dataframe
for locus_name in sorted(set(train_size_df["locus"])):
    plt.figure(figsize=(10,5))
    train_size_plot = sns.pointplot(x="train_nb", y="accuracy", hue="method", data=train_size_df[train_size_df.locus == locus_name])
    for item in train_size_plot.get_xticklabels():
        item.set_rotation(90)
    plt.gcf().suptitle("Accuracy by size of training set for {}".format(locus_name))
    plt.show()

for locus_name in sorted(set(train_size_df["locus"])):
    plt.figure(figsize=(10,5))
    train_size_plot = sns.pointplot(x="train_nb", y="accuracy", hue="method", data=train_size_df[(train_size_df.locus == locus_name) & (train_size_df.method == "C-Support Vector")])
    for item in train_size_plot.get_xticklabels():
        item.set_rotation(90)
    plt.gcf().suptitle("Accuracy by size of training set for {}".format(locus_name))
    plt.show()

In [ ]:
for locus_name in sorted(set(train_size_df["locus"])):
    test = accuracy_df[accuracy_df.locus == locus_name].apply(getTrainSize, axis=1)
    print(locus_name, set(test))

##### Accuracy and number of reads in sample

In [ ]:
# Classify data with ShuffleSplit
accuracy_reads_locus_df = []
for locus in models_param:
    for nb_reads_threshold in range(50, 550, 50):
        # Classifiers
        used_classifiers = [
            LocusClassifier(
                locus["id"], "C-Support Vector", SVC(random_state=init_random_seed, probability=True), data_method_name=method_name
            )
        ]
        # Select data
        whole_labels = []
        whole_data_obj = []
        for spl in msi_eval:
            if spl.loci[locus["id"]].results[method_name].getNbFrag() * 2 >= nb_reads_threshold:
                diamic_name = spl.name.replace("_L001", "").split("_")[0]
                if diamic_name in status_by_spl:
                    status = status_by_spl[diamic_name][locus["name"]]
                    if status != "Undetermined":
                        whole_labels.append(status)
                        whole_data_obj.append(spl)
                        spl.loci[locus["id"]].results["model"] = LocusResPairsCombi(status, data=spl.loci[locus["id"]].results[method_name].data)
        # Calculates results
        cv = ShuffleSplit(n_splits=200, test_size=0.4, random_state=init_random_seed)
        res_df = cross_val_score_custom(locus, whole_data_obj, whole_labels, used_classifiers, cv)
        res_df["nb_reads"] = nb_reads_threshold
        accuracy_reads_locus_df.append(res_df)
accuracy_reads_df = pd.concat(accuracy_reads_locus_df)

# Plot accuracy
accuracy_reads_plt = sns.factorplot(x="locus", y="accuracy", hue="nb_reads", data=accuracy_reads_df, kind="bar", size=7)
for ax in accuracy_reads_plt.axes.flat:
    for item in ax.patches:
        height = 0 if np.isnan(item.get_height()) else item.get_height()
        ax.annotate(
            '{:.1%}'.format(item.get_height()),
            (
                item.get_x() + item.get_width() / 2.,
                item.get_height()
            ),
            ha='center', va='center', fontsize=11, color='gray', rotation=90, xytext=(0, 20),
            textcoords='offset points')
    ax.set_ylim(0.92, 1.0)
plt.subplots_adjust(top=0.9)
plt.gcf().suptitle("Classification accuracy")
plt.show()

#### 2- Classification accuracy with model used in INCa-v2

On calcule le taux d'erreur de classification pour chaque classifieur et chaque marqueur en prenant pour base d'entrainement commune le premier modèle utilisé pour l'INCa-v2.

In [ ]:
# Get current model
data_folder = os.path.join(os.getcwd(), "data")
inca_v2_model = MSIReport.parse(os.path.join(data_folder, "models_TSCA_INCa_V2_2", "INCa-V2_models.json"))

# Display samples
rows = []
for spl in inca_v2_model:
    diamic_name = spl.name.replace("_L001", "").split("_")[0]
    rows.append({"sample": spl.name})
    for locus, status in status_by_spl[diamic_name].items():
        locus = locus.replace("MSI_", "") + "_status"
        rows[-1][locus] = status
pd.DataFrame(rows)

In [ ]:
# Results
models_names = {}
cols = {"sample": [], "locus": [], "method": [], "score": [], "expected": [], "predicted": [], "prediction_status": []}
for locus in models_param:
    # Classifiers
    used_classifiers = [
        LocusClassifier(
            locus["id"], "Logistic Regression", LogisticRegression(random_state=init_random_seed), data_method_name=method_name
        ),
        LocusClassifier(
            locus["id"], "Decision Tree", DecisionTreeClassifier(random_state=init_random_seed), data_method_name=method_name
        ),
        LocusClassifier(
            locus["id"], "K-Neighbors", KNeighborsClassifier(2), data_method_name=method_name
        ),
        LocusClassifier(
            locus["id"], "Gaussian Naive Bayes", GaussianNB(), data_method_name=method_name
        ),
        LocusClassifier(
            locus["id"], "C-Support Vector", SVC(random_state=init_random_seed, probability=True), data_method_name=method_name
        ),
        LocusClassifier(
            locus["id"], "Random Forest", RandomForestClassifier(random_state=init_random_seed), data_method_name=method_name
        ),
        LocusClassifier(
            locus["id"], "Random Forest 20", RandomForestClassifier(random_state=init_random_seed, n_estimators=20), data_method_name=method_name
        ),
        LocusClassifier(
            locus["id"], "Random Forest 30", RandomForestClassifier(random_state=init_random_seed, n_estimators=30), data_method_name=method_name
        )
    ]
    # sklearn
    for idx, classifier in enumerate(used_classifiers):
        # Creates model labels
        train_data = []
        for spl in inca_v2_model:
            diamic_name = spl.name.replace("_L001", "").split("_")[0]
            locus_res = spl.loci[locus["id"]].results
            if "model" in locus_res and locus_res["model"].status != "Undetermined":
                if locus_res["model"].getNbFrag() * 2 >= min_nb_reads:
                    models_names[spl.name] = 1
                    status = status_by_spl[diamic_name][locus["name"]]
                    if status != locus_res["model"].status:
                        print("The expected status for the sample {} is {} on locus {} but unfortunately the status {} is found in model.".format(
                            spl.name, status, locus["name"], locus_res["model"].status
                        ))
                    train_data.append(spl)
        # Train
        classifier.fit(train_data)
        # Classify
        for spl in msi_eval:
            diamic_name = spl.name.replace("_L001", "").split("_")[0]
            if diamic_name in status_by_spl and spl.name not in models_names:
                if spl.loci[locus["id"]].results[method_name].getNbFrag() * 2 >= min_nb_reads:
                    expected_status = status_by_spl[diamic_name][locus["name"]]
                    if expected_status != "Undetermined":
                        predicted_status = classifier.predict([spl])[0]
                        cols["sample"].append(spl.name.replace("_L001", ""))
                        cols["locus"].append(locus["name"])
                        cols["method"].append(classifier.method_name)
                        cols["score"].append(classifier._get_scores([predicted_status])[0])
                        cols["expected"].append(expected_status)
                        cols["predicted"].append(predicted_status)
                        if predicted_status != "Undetermined":
                            cols["prediction_status"].append(expected_status == predicted_status)
                        else:
                            cols["prediction_status"].append("NA")
    # MIAmS
    for idx, classifier_name in enumerate(["MSINGS", method_name]):
        for spl in msi_eval:
            diamic_name = spl.name.replace("_L001", "").split("_")[0]
            if diamic_name in status_by_spl and spl.name not in models_names:
                if spl.loci[locus["id"]].results[method_name].getNbFrag() * 2 >= min_nb_reads:
                    expected_status = status_by_spl[diamic_name][locus["name"]]
                    if expected_status != "Undetermined":
                        predicted_status = spl.loci[locus["id"]].results[classifier_name].status
                        cols["sample"].append(spl.name.replace("_L001", ""))
                        cols["locus"].append(locus["name"])
                        cols["method"].append(classifier_name)
                        cols["score"].append(spl.loci[locus["id"]].results[classifier_name].score)
                        cols["expected"].append(expected_status)
                        cols["predicted"].append(predicted_status)
                        if predicted_status != "Undetermined":
                            cols["prediction_status"].append(expected_status == predicted_status)
                        else:
                            cols["prediction_status"].append("NA")
eval_df = pd.DataFrame(data=cols)

In [ ]:
# Plot evaluation
plt_whole = sns.countplot(x="prediction_status", hue="method", data=eval_df, order=[True, False, "NA"])
plt_whole.set_title("Locus classification")
plt.legend(loc='upper right')
plt.show()

plt_by_locus = sns.factorplot(x="prediction_status", hue="method", col="locus", data=eval_df, order=[True, False, "NA"], kind="count")
for ax in plt_by_locus.axes.flat:
    locus_name = ax.get_title().split(" ")[-1]
    total_count = len(eval_df[(eval_df["locus"] == locus_name) & (eval_df["method"] == "Logistic Regression")])
    for item in ax.patches:
        height = 0 if np.isnan(item.get_height()) else item.get_height()
        ax.annotate(
            '{:.1%}'.format(item.get_height()/total_count),
            (
                item.get_x() + item.get_width() / 2.,
                item.get_height()
            ),
            ha='center', va='center', fontsize=11, color='gray', rotation=90, xytext=(0, 20),
            textcoords='offset points')
plt.subplots_adjust(top=0.8)
plt.gcf().suptitle("Locus classification")
plt.show()

Les classification avec Logistic Regression, K-Neighbors et C-Support Vector ont les plus faibles taux d'erreur sur l'ensemble des marqueurs.

Les bonnes performances de classifieurs pourtant très simples comme le Logistic Regression et K-Neighbors s'expliquent par le peu de différence intra-statut et la forte différence inter-statut dans les profils des échantillons fournis. L'ensemble de test ne contient pas de cas difficile en terme de % de séquences dans le deuxième pic ou de distance entre les pics.

mSINGS est toujours moins bon que ces trois classieurs (excepté sur BAT25 en comparaison avec la régression logistique). Il a le taux d'erreur le plus élevé sur BAT26.

#### 3- Prediction probabilities evaluation

On cherche à savoir si les classifieurs fournissent une probabilité de prédiction utilisable. C'est à dire un score qui permet de distinguer 2 groupes:
  * Un groupe contenant la très grande majorité des fausses prédiction (idéalement toutes) et quelques bonnes prédictions.
  * Un groupe dont on est sûr de la prédiction (peu voire pas de mauvaise prédiction et un maximum de bonnes prédictions).

Un classifieur qui à un très faible taux d'erreur mais un score de probabilité ne permettant pas d'éliminer les fausses prédictions ne sera pas forcément préféré à un classifieur avec un faible taux d'erreur mais pour lequel on pourra faire confiance au score de prédiction. Dans le premier cas on aura jamais confiance dans la prédiction et on sera obligé de contrôler tous les cas. Dans le deuxième cas on fera confiance dans la classification des très haut score et on reverra manuellement tout les autres.

In [ ]:
# Evaluate prediction probability
clf_names = ["Logistic Regression", "Decision Tree", "K-Neighbors", "C-Support Vector", "Gaussian Naive Bayes", "Random Forest", "Random Forest 20", "Random Forest 30", "MSINGS", method_name]
for clf_name in clf_names:
    sns.factorplot(
        x="prediction_status",
        y="score",
        col="locus",
        order=[True, False],
        data=eval_df[eval_df.method == clf_name],
        kind="box"
    )
    plt.subplots_adjust(top=0.9)
    plt.gcf().suptitle(clf_name) # can also get the figure from plt.gcf()
    plt.show()

Les probabilités associées à la prédiction du Random Forest et dans une moindre mesure du C-Support Vector semblent permettre de distinguer la très grande majorité des mauvaises classification (au prix bien entendu de perdre une part importante de bonne classification).

Attention : Dans le jeux de données le nombre de fausses classification n'était pas très important et il est donc difficile de tirer des conclusions indiscutable. on travaillera sur des tendance que l'on confirmera en simulation ou avec plus de données.

In [ ]:
# Evaluate prediction probability usage
prediction_score_threshold = 0.9
methods = ["Random Forest", "C-Support Vector"]
for clf_name in methods:
    labels = ["Score", "Valid prediction", "Invalid prediction"]
    values = [
        (
            ">= {}".format(prediction_score_threshold),
            len(eval_df[(eval_df.prediction_status == True) & (eval_df.method == clf_name) & (eval_df.score >= prediction_score_threshold)]),
            len(eval_df[(eval_df.prediction_status == False) & (eval_df.method == clf_name) & (eval_df.score >= prediction_score_threshold)])
        ),
        (
            "< {}".format(prediction_score_threshold),
            len(eval_df[(eval_df.prediction_status == True) & (eval_df.method == clf_name) & (eval_df.score < prediction_score_threshold)]),
            len(eval_df[(eval_df.prediction_status == False) & (eval_df.method == clf_name) & (eval_df.score < prediction_score_threshold)])
        )
    ]
    print("{}\n".format(clf_name))
    print(pd.DataFrame.from_records(values, columns=labels))
    print("")

Avec une probabilité de prédiction >= {{threshold}} on élimine quasiment toutes les fausses prédictions et on obtiens un nombre suffisant de classification automatique.

On constate que si on se concentre uniquement sur les prédiction avec une forte confiance le Random forest deviens meilleur que le C-Support Vector.

Au vu de ces chiffres on pourrait utiliser le Random Forest pour donner les prédictions de confiance et le C-Support Vector pour aider à la décision sur la revue manuelle des autres cas.

In [ ]:
# List errors with high prediction score
eval_df[(eval_df.prediction_status == False) & eval_df.method.isin(methods) & (eval_df.score >= prediction_score_threshold)]

### Evaluation at sample level

#### 1- Status based on pentaplex

In [ ]:
# Classify samples with same models in all methods and votation of pentaplex
nb_spl = 0
cols = {"sample": [], "method": [], "nb_loci": [], "nb_MSS": [], "nb_MSI": [], "score": [], "expected": [], "predicted": [], "prediction_status": []}
for clf_name in clf_names:
    filtered_df = eval_df[(eval_df.method == clf_name) & (eval_df.locus != "MSI_HT17")]
    res_by_spl = {}
    for index, row in filtered_df.iterrows():
        if row["sample"] not in res_by_spl:
            res_by_spl[row["sample"]] = {}
        res_by_spl[row["sample"]][row["locus"]] = row["predicted"]
    for spl, loci in res_by_spl.items():
        diamic_name = spl.replace("_L001", "").split("_")[0]
        status_list = list(loci.values())
        nb_MSS = status_list.count("MSS")
        nb_MSI = status_list.count("MSI")
        expected_spl_status = status_by_spl[diamic_name]["sample"]
        predicted_spl_status = "Undetermined"
        if nb_MSS > nb_MSI and nb_MSS >= 3:
            predicted_spl_status = "MSS"
        elif nb_MSI > nb_MSS and nb_MSI >= 3:
            predicted_spl_status = "MSI"
        cols["sample"].append(spl)
        cols["method"].append(clf_name)
        cols["score"].append(np.nan)
        cols["nb_loci"].append(nb_MSS + nb_MSI)
        cols["nb_MSS"].append(nb_MSS)
        cols["nb_MSI"].append(nb_MSI)
        cols["expected"].append(expected_spl_status)
        cols["predicted"].append(predicted_spl_status)
        if expected_spl_status == "Undetermined":
            cols["prediction_status"].append("DB_Undetermined")
        elif predicted_spl_status != "Undetermined":
            cols["prediction_status"].append(expected_spl_status == predicted_spl_status)
        else:
            cols["prediction_status"].append("NA")
    nb_spl = len(res_by_spl)
spl_pentaplex_df = pd.DataFrame(data=cols)

# Plot evaluation
ax = sns.countplot(x="prediction_status", hue="method", data=spl_pentaplex_df, order=[True, False, "NA", "DB_Undetermined"])
ax.set_ylim(0, 250)
for item in ax.patches:
    height = 0 if np.isnan(item.get_height()) else item.get_height()
    ax.annotate(
        '{:.1%}'.format(item.get_height()/nb_spl),
        (
            item.get_x() + item.get_width() / 2.,
            item.get_height()
        ),
        ha='center', va='center', fontsize=11, color='gray', rotation=90, xytext=(0, 20),
        textcoords='offset points')
ax.set_title("Samples stability classification based on pentaplex")
plt.show()

# List or errors
spl_pentaplex_df[(spl_pentaplex_df.prediction_status == False) & (spl_pentaplex_df.method == "C-Support Vector")]

Les taux d'erreur avec le vote sur les 5 marqueurs du pentaplex sont faibles : de 1 à 2 % pour les classifieurs déjà retenus K-neighbors et C-Support Vector.

Une partie faible mais significative des échantillons n'a pas de statut (4 à 5 % de NA).

#### 2- Status based on HT17

In [ ]:
# Filter data for HT17 and compare to sample
cols = {"sample": [], "method": [], "score": [], "expected": [], "predicted": [], "prediction_status": []}
for curr_method in clf_names:
    filtered_df = eval_df[(eval_df.method == curr_method) & (eval_df.locus == "MSI_HT17")]
    predict_by_spl = {}
    score_by_spl = {}
    for index, row in filtered_df.iterrows():
        predict_by_spl[row["sample"]] = row["predicted"]
        score_by_spl[row["sample"]] = row["score"]
    for spl, predicted_ht17 in predict_by_spl.items():
        diamic_name = spl.replace("_L001", "").split("_")[0]
        expected_spl_status = status_by_spl[diamic_name]["sample"]
        cols["sample"].append(spl)
        cols["method"].append(curr_method)
        cols["score"].append(score_by_spl[spl])
        cols["expected"].append(expected_spl_status)
        cols["predicted"].append(predicted_ht17)
        if expected_spl_status == "Undetermined":
            cols["prediction_status"].append("DB_Undetermined")
        elif predicted_ht17 != "Undetermined":
            cols["prediction_status"].append(expected_spl_status == predicted_ht17)
        else:
            cols["prediction_status"].append("NA")
spl_ht17_df = pd.DataFrame(data=cols)

# Plot evaluation
nb_samples = len(spl_ht17_df[spl_ht17_df.method == "Random Forest"])
ax = sns.countplot(x="prediction_status", hue="method", data=spl_ht17_df, order=[True, False, "NA", "DB_Undetermined"])
ax.set_ylim(0, 250)
for item in ax.patches:
    height = 0 if np.isnan(item.get_height()) else item.get_height()
    ax.annotate(
        '{:.1%}'.format(height/nb_samples),
        (
            item.get_x() + item.get_width() / 2.,
            item.get_height()
        ),
        ha='center', va='center', fontsize=11, color='gray', rotation=90, xytext=(0, 20),
        textcoords='offset points')
ax.set_title("Samples stability classification based on HT17")
plt.legend(loc='upper center')
plt.show()

# List or errors
spl_ht17_df[(spl_ht17_df.prediction_status == False) & (spl_ht17_df.method == "C-Support Vector")]

Le classifieurs basés Logistic Regression, K-neighbors et C-Support Vector ont de bien meilleur résultats que mSINGS: 2.1% d'erreur contre 12.9%.

Bien que le taux d'erreur soit plus fort qu'avec le pentaplex (2.1% contre 1%), le taux de correctement classifiés est supérieur car on a moins de problème avec ce seul marqueur (97.9% contre 94.5%).

In [ ]:
# Filter data by locus and compare to sample
for curr_model in models_param:
    cols = {"sample": [], "method": [], "expected": [], "predicted": [], "prediction_status": []}
    for curr_method in clf_names:
        filtered_df = eval_df[(eval_df.method == curr_method) & (eval_df.locus == curr_model["name"])]
        predict_by_spl = {}
        for index, row in filtered_df.iterrows():
            predict_by_spl[row["sample"]] = row["predicted"]
        for spl, predicted_locus in predict_by_spl.items():
            diamic_name = spl.replace("_L001", "").split("_")[0]
            expected_spl_status = status_by_spl[diamic_name]["sample"]
            if expected_spl_status != "Undetermined":
                cols["sample"].append(spl)
                cols["method"].append(curr_method)
                cols["expected"].append(expected_spl_status)
                cols["predicted"].append(predicted_locus)
                if expected_spl_status == "Undetermined":
                    cols["prediction_status"].append("DB_Undetermined")
                elif predicted_locus != "Undetermined":
                    cols["prediction_status"].append(expected_spl_status == predicted_locus)
                else:
                    cols["prediction_status"].append("NA")
    locus_predict_spl_df = pd.DataFrame(data=cols)

    # Plot evaluation
    nb_samples = len(locus_predict_spl_df[locus_predict_spl_df.method == "Random Forest"])
    ax = sns.countplot(x="prediction_status", hue="method", data=locus_predict_spl_df, order=[True, False, "NA", "DB_Undetermined"])
    ax.set_ylim(0, 250)
    for item in ax.patches:
        height = 0 if np.isnan(item.get_height()) else item.get_height()
        ax.annotate(
            '{:.1%}'.format(item.get_height()/nb_samples),
            (
                item.get_x() + item.get_width() / 2.,
                item.get_height()
            ),
            ha='center', va='center', fontsize=11, color='gray', rotation=90, xytext=(0, 20),
            textcoords='offset points')
    ax.set_title("Samples stability classification based on {}".format(curr_model["name"]))
    plt.legend(loc='upper center')
    plt.show()

Sur les cas évalués quand NR27 est présent (142 librairies) il est meilleur que HT17 (194 librairies) pour donner le status de l'échantillon.

In [ ]:
# Errors with C-Support Vector
spl_ht17_df[(spl_ht17_df.method == "C-Support Vector") & (spl_ht17_df.prediction_status == False)]

Les profils de 17T044302_S22 et 17T005560_S47 correspondent parfaitement à du MSS.

Les deux autres ont un score plus réduit ce qui va dans le sens d'une erreur avec une faible confiance.

In [ ]:
# Errors with Random Forest
spl_ht17_df[(spl_ht17_df.method == "Random Forest") & (spl_ht17_df.prediction_status == False)]

Excepté pour 17T044302_S22 et 17T005560_S47 le score montre que l'on pourrait mettre une alerte sur les erreurs de classification.

In [ ]:
############## combiner pentaplex + HT17 pour voir si erreur diminue

## Samples database information

Les données sont de 3 sources:
 * les résultats extrait de diamic pour l'électrophorèse sur le pentaplex : BAT25, BAT26, NR21, NR22, NR24. Ces résultats ont été complétés pour les échantillons passés en NGS par une révision manuelle : NR27 et HT17.
 * les résultats révisé manuellement des échantillons de routine à prioris MSS passés en NGS INCa-V2 pour valider la partie mutation du panel :  HT17 NR27
 * les résultats des échantillons du run NGS de montpellier 180430 révisé manuellement: BAT25, BAT26, NR21, NR22, NR27 et HT17. Le run contenait des MSS connu, des MSI connu et des statut indéterminé (passé pour les mutations).

Les révisions manuelles faite via NGS s'appuient sur les profils des combinaisons de paire en tenant compte des réplicats.


In [ ]:
# Load status by locus and compare to the sample status
cols = {"sample": [], "in_ngs": [], "sample_status": [], "locus": [], "classification_status": []}
for spl_name, status in status_by_spl.items():
    in_ngs = False
    if status["MSI_HT17"] != "" and status["sample"]:
        in_ngs = True
    for locus in ["MSI_BAT25", "MSI_BAT26", "MSI_NR21", "MSI_NR22", "MSI_NR24", "MSI_NR27", "MSI_HT17"]:
        cols["sample"].append(spl_name)
        cols["in_ngs"].append(in_ngs)
        cols["sample_status"].append(status["sample"])
        cols["locus"].append(locus)
        validity = "NA"
        if status["sample"] != "Undetermined" and status[locus] != "" and status[locus] != "Undetermined":
            validity = (status["sample"] == status[locus])
        cols["classification_status"].append(validity)
locus_status_df = pd.DataFrame(data=cols)

In [ ]:
# Plot the number of samples well classified if the reference is only one locus
nb_spl = len(locus_status_df[locus_status_df.locus == "MSI_HT17"])
ax = sns.countplot(x="classification_status", hue="locus", data=locus_status_df, order=[True, False, "NA"])
ax.set_ylim(0, nb_spl + nb_spl * 0.2)
for item in ax.patches:
    height = 0 if np.isnan(item.get_height()) else item.get_height()
    ax.annotate(
        '{:.1%}'.format(item.get_height()/nb_spl),
        (
            item.get_x() + item.get_width() / 2.,
            item.get_height()
        ),
        ha='center', va='center', fontsize=11, color='gray', rotation=90, xytext=(0, 20),
        textcoords='offset points')
ax.set_title("Number of samples well classified by only one locus")
plt.legend(loc='upper center')
plt.show()

Le taux d'erreur si l'on se fiait à uniquement un marqueur est inférieur à 2%.

In [ ]:
# Plot the number of samples well classified if the reference is only one locus in samples selected for NGS
ngs_locus_status_df = locus_status_df[locus_status_df.in_ngs == True]
nb_spl = len(ngs_locus_status_df[ngs_locus_status_df.locus == "MSI_HT17"])
ax = sns.countplot(x="classification_status", hue="locus", data=ngs_locus_status_df, order=[True, False, "NA"])
ax.set_ylim(0, nb_spl + nb_spl * 0.2)
for item in ax.patches:
    height = 0 if np.isnan(item.get_height()) else item.get_height()
    ax.annotate(
        '{:.1%}'.format(item.get_height()/nb_spl),
        (
            item.get_x() + item.get_width() / 2.,
            item.get_height()
        ),
        ha='center', va='center', fontsize=11, color='gray', rotation=90, xytext=(0, 20),
        textcoords='offset points')
ax.set_title("Number of samples selected for NGS well classified by only one locus")
plt.legend(loc='upper center')
plt.show()

#
locus_status_df[(locus_status_df.classification_status == False) & (locus_status_df.locus == "MSI_HT17")]

Sur les cas utilisés pour l'analyse HT17 à l'un des plus fort pouvoir prédictif. Il reste cependant au niveau de BAT26 et NR22.

## Learn on simulated data

### 1- Create simulated data

In [ ]:
# Simulate data
nb_val = 10000
nb_repeat = 20

for curr_model in models_param:
    center = curr_model["mean"]
    std_dev = [curr_model["std_dev"] - 0.1, curr_model["std_dev"], curr_model["std_dev"] + 0.1]

    np.random.seed(init_random_seed)
    distances = range(1, 10)
    tum_frequencies = range(0, 100, 5)
    nb_spl = nb_repeat * len(distances) * len(tum_frequencies)
    random_seeds = rd.sample(range(1, 2**32-1), k=nb_spl + 1)
    curr_idx = 0
    for curr_generator in range(nb_repeat):
        for curr_af in tum_frequencies:   
            for curr_distance in distances:
                nb_tumor = int((curr_af/100)*nb_val)
                nb_healthy = nb_val - nb_tumor
                series_params = [
                    {"random_seed": random_seeds[curr_idx], "center": center, "std_dev": rd.choice(std_dev), "nb_val": nb_healthy},
                    {"random_seed": random_seeds[curr_idx] + 1, "center": center - curr_distance, "std_dev": rd.choice(std_dev), "nb_val": nb_tumor},
                ]

                # Create sub-series
                series = list()
                for curr_params in series_params:
                    if "random_seed" in curr_params:
                        np.random.seed(curr_params["random_seed"])
                    series.append(
                        np.random.normal(curr_params["center"], curr_params["std_dev"], curr_params["nb_val"]).astype(int)
                    )

                # Next random seed
                curr_idx = curr_idx + 1

                # Add loci to evaluated
                spl_name = "tum_{}_on_{}".format(curr_af, center - curr_distance)
                merged_series = mergeSeries(series)
                loci_res = LocusResPairsCombi(None, None, {"nb_by_length": getStacked(merged_series)})
                loci = MSILocus(curr_model["id"], curr_model["name"], {method_name: loci_res})
                curr_model["simulated_data"].append(
                    MSISample(spl_name, {curr_model["id"]: loci})
                )

In [ ]:
# Store simulation data
simu_folder = os.path.join(os.getcwd(), "data", "simulation")
for curr_model in models_param:
    MSIReport.write(
        curr_model["simulated_data"],
        os.path.join(simu_folder, curr_model["name"] + ".json")
    )

### 2- Classify simulated samples with learn on simulated samples

In [ ]:
classifiers_name = ["Logistic Regression", "Decision Tree", "K-Neighbors", "Gaussian Naive Bayes", "C-Support Vector", "Random Forest", "Random Forest 20", "Random Forest 30"]

In [ ]:
# Classify simulated samples with learn on simulated samples
cols = {
    "sample": [],
    "locus": [],
    "method": [],
    "training_idx": [],
    "score": [],
    "expected": [],
    "predicted": [],
    "prediction_status": []
}

summary_cols = {
    "locus": [],
    "method": [],
    "accuracy": []
}

for curr_model in models_param:
    # Creates labels
    whole_name = []
    whole_data = []
    whole_labels = []
    for spl in curr_model["simulated_data"]:
        if spl.loci[curr_model["id"]].results[method_name].getNbFrag() * 2 >= min_nb_reads:
            series = spl.loci[curr_model["id"]].results[method_name].getDensePrct(50, 200)
            status = "MSI"
            if spl.name.startswith("tum_0_") or spl.name.startswith("tum_5_") or \
               spl.name.endswith("_on_{}".format(int(curr_model["mean"]))) or \
               spl.name.endswith("_on_{}".format(int(curr_model["mean"]) - 1)):
                status = "MSS"
            whole_name.append(spl.name)
            whole_data.append(series)
            whole_labels.append(status)
    
    # Calculates results
    training_idx = 0
    cv = ShuffleSplit(n_splits=200, test_size=0.25, random_state=init_random_seed)
    print("{}: {} MSS, {} MSI, {} datasets".format(curr_model["name"], whole_labels.count("MSS"), whole_labels.count("MSI"), cv.get_n_splits()))
    for train_idx, test_idx in cv.split(whole_data, groups=whole_labels):
        training_idx += 1
        # Split dataset in train and test sub-datasets
        train = {"names": [], "data": [], "labels": []}
        test = {"names": [], "data": [], "labels": []}
        for data_idx, data_series in enumerate(whole_data):
            if data_idx in train_idx:
                train["names"].append(whole_name[data_idx])
                train["data"].append(whole_data[data_idx])
                train["labels"].append(whole_labels[data_idx])
            elif data_idx in test_idx:
                test["names"].append(whole_name[data_idx])
                test["data"].append(whole_data[data_idx])
                test["labels"].append(whole_labels[data_idx])
        # Classifiers
        learning_methods = [
            {"name": "Logistic Regression", "clf": LogisticRegression(random_state=init_random_seed)},
            {"name": "Decision Tree", "clf": DecisionTreeClassifier(random_state=init_random_seed)},
            {"name": "K-Neighbors", "clf": KNeighborsClassifier(2)},
            {"name": "Gaussian Naive Bayes", "clf": GaussianNB()},
            {"name": "C-Support Vector", "clf": SVC(random_state=init_random_seed, probability=True)},
            {"name": "Random Forest", "clf": RandomForestClassifier(random_state=init_random_seed)},
            {"name": "Random Forest 20", "clf": RandomForestClassifier(random_state=init_random_seed, n_estimators=20)},
            {"name": "Random Forest 30", "clf": RandomForestClassifier(random_state=init_random_seed, n_estimators=30)},
        ]
        # Use classifiers
        for idx, curr_method in enumerate(learning_methods):
            # Train
            clf = curr_method["clf"].fit(train["data"], train["labels"])
            # Classify
            predictions = clf.predict(test["data"])
            probabilities = clf.predict_proba(test["data"])
            nb_true = 0
            nb_false = 0
            for pred_idx, pred_label in enumerate(predictions):
                cols["sample"].append(test["names"][pred_idx])
                cols["locus"].append(curr_model["name"])
                cols["method"].append(curr_method["name"])
                cols["training_idx"].append(training_idx)
                status_idx = np.nonzero(clf.classes_ == pred_label)[0][0]
                cols["score"].append(probabilities[pred_idx][status_idx])
                cols["expected"].append(test["labels"][pred_idx])
                cols["predicted"].append(pred_label)
                cols["prediction_status"].append(test["labels"][pred_idx] == pred_label)
                if test["labels"][pred_idx] == pred_label:
                    nb_true += 1
                else:
                    nb_false += 1
            summary_cols["locus"].append(curr_model["name"])
            summary_cols["method"].append(curr_method["name"])
            summary_cols["accuracy"].append(nb_true/(nb_true + nb_false))
        
simuTrain_simuData_df = pd.DataFrame(data=cols)
summary_simuTrain_simuData_df = pd.DataFrame(data=summary_cols)

In [ ]:
# Plot zoomed accuracy
accuracy_plt = sns.factorplot(x="locus", y="accuracy", hue="method", data=summary_simuTrain_simuData_df, kind="bar", size=6)
for ax in accuracy_plt.axes.flat:
    ax.set_ylim(0.95, 1.0)
plt.subplots_adjust(top=0.95)
plt.gcf().suptitle("Zoomed classification accuracy")
plt.show()

# Plot accuracy boxplot
accuracy_plt = sns.boxplot(x="locus", y="accuracy", hue="method", data=summary_simuTrain_simuData_df, medianprops=dict(linewidth=2, color='firebrick'))
plt.subplots_adjust(top=0.95)
plt.gcf().suptitle("Classification accuracy")
plt.show()

# Plot accuracy boxplot
accuracy_plt = sns.boxplot(x="locus", y="accuracy", hue="method", data=summary_simuTrain_simuData_df, medianprops=dict(linewidth=2, color='firebrick'))
accuracy_plt.set_ylim(0.96, 1.0)
plt.subplots_adjust(top=0.95)
plt.gcf().suptitle("Zoomed classification accuracy")
plt.show()

In [ ]:
# Evaluate prediction probability
for curr_method in classifiers_name:
    sns.factorplot(
        x="prediction_status",
        y="score",
        col="locus",
        order=[True, False],
        data=simuTrain_simuData_df[simuTrain_simuData_df.method == curr_method],
        kind="box"
    )
    plt.subplots_adjust(top=0.9)
    plt.gcf().suptitle("Prediction probability by prediction status with {}".format(curr_method))
    plt.show()

La forte utilité de la probabilité de prédction du Random Forest et dans une moindre mesure du C-Support Vector sont confirmée.

On confirme également que plus on augmente le nombre d'abre du Random Forest plus l'accuracy augmente mais moins la probabilité de la prédiction est utilisable.

### 3- Classify real samples with learn on simulated samples

In [ ]:
# Classify and compare to the expected
cols = {"sample": [], "locus": [], "method": [], "expected": [], "predicted": [], "prediction_status": []}
for curr_model in models_param:
    for idx, curr_method in enumerate(learning_methods):
        # Creates simulation labels
        whole_data = []
        whole_labels = []
        for spl in curr_model["simulated_data"]:
            if spl.loci[curr_model["id"]].results[method_name].getNbFrag() * 2 >= min_nb_reads:
                series = spl.loci[curr_model["id"]].results[method_name].getDensePrct(50, 200)
                status = "MSI"
                if spl.name.startswith("tum_0_") or spl.name.startswith("tum_5_") or \
                   spl.name.endswith("_on_{}".format(int(curr_model["mean"]))) or \
                   spl.name.endswith("_on_{}".format(int(curr_model["mean"]) - 1)):
                    status = "MSS"
                whole_data.append(series)
                whole_labels.append(status)
        # Train
        clf = curr_method["clf"].fit(whole_data, whole_labels)
        # Classify
        for spl in msi_eval:
            diamic_name = spl.name.replace("_L001", "").split("_")[0]
            if diamic_name in status_by_spl:
                if spl.loci[curr_model["id"]].results[method_name].getNbFrag() * 2 >= min_nb_reads:
                    series = spl.loci[curr_model["id"]].results[method_name].getDensePrct(50, 200)
                    predicted_status = clf.predict([series])[0]
                    expected_status = status_by_spl[diamic_name][curr_model["name"]]
                    cols["sample"].append(spl.name.replace("_L001", ""))
                    cols["locus"].append(curr_model["name"])
                    cols["method"].append(curr_method['name'])
                    cols["expected"].append(expected_status)
                    cols["predicted"].append(predicted_status)
                    if expected_status != "Undetermined" and predicted_status != "Undetermined":
                        cols["prediction_status"].append(expected_status == predicted_status)
                    else:
                        cols["prediction_status"].append("NA")
simuTrain_realData_df = pd.DataFrame(data=cols)

In [ ]:
# Plot evaluation
plt_whole = sns.countplot(x="prediction_status", hue="method", data=simuTrain_realData_df, order=[True, False, "NA"])
plt_whole.set_title("Locus classification")

plt_by_locus = sns.factorplot(x="prediction_status", hue="method", col="locus", data=simuTrain_realData_df, order=[True, False, "NA"], kind="count")
for ax in plt_by_locus.axes.flat:
    locus_name = ax.get_title().split(" ")[-1]
    total_count = len(simuTrain_realData_df[(simuTrain_realData_df.locus == locus_name) & (simuTrain_realData_df.method == "Random Forest")])
    for item in ax.patches:
        height = 0 if np.isnan(item.get_height()) else item.get_height()
        ax.annotate(
            '{:.1%}'.format(item.get_height()/total_count),
            (
                item.get_x() + item.get_width() / 2.,
                item.get_height()
            ),
            ha='center', va='center', fontsize=11, color='gray', rotation=90, xytext=(0, 20),
            textcoords='offset points')
plt.subplots_adjust(top=0.8)
plt.gcf().suptitle("Locus classification")
plt.show()

In [ ]:
error_df = eval_df[eval_df["prediction_status"] == False]
g = sns.factorplot(x="sample", hue="locus", data=error_df, palette="muted", kind='count', aspect=3)
g.set_xticklabels(rotation=90)
g.set_ylabels("Nb_methods")

## Analysis without machine learning

In [ ]:
from scipy.signal import savgol_filter, find_peaks_cwt
from scipy.ndimage.filters import gaussian_filter1d

smooth_sigma = 1.1

test_curves = [
    {"data": [0, 100, 0, 0, 0, 0], "nb_peaks": 1},
    {"data": [100, 0, 0, 0, 0], "nb_peaks": 1},
    {"data": [0, 100, 80, 90, 70, 0], "nb_peaks": 1},
    {"data": [0, 100, 80, 75, 70, 65, 60, 55, 50, 45, 40, 35, 30, 25, 20, 15, 10, 5, 0], "nb_peaks": 1},
    {"data": [0, 100, 0, 40, 0], "nb_peaks": 1},
    {"data": [0, 100, 0, 0, 40, 0], "nb_peaks": 2},
    {"data": [0, 30, 80, 100, 0, 0, 20, 40, 20, 0], "nb_peaks": 2},
    {"data": [0, 100, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 17, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 0], "nb_peaks": 2},   
]

def getPeaks(smoothed_profile, max_noise=0.05):
    len_smoothed_profile = len(smoothed_profile)

    # Find peaks
    peaks_idx = []
    for prv_idx, val in enumerate(smoothed_profile[1:]):
        if val > smoothed_profile[prv_idx] and (prv_idx + 3 > len_smoothed_profile or val >= smoothed_profile[prv_idx + 2]):
            peaks_idx.append(prv_idx + 1)

    # Retrieve weight of peaks 
    peaks_wheight = {idx: smoothed_profile[idx] for idx in peaks_idx}
    for peak_idx in peaks_idx:
        idx = peak_idx
        is_desc = True
        while is_desc:
            if idx + 1 < len_smoothed_profile and smoothed_profile[idx] >= smoothed_profile[idx + 1] and smoothed_profile[idx] != 0:
                peaks_wheight[peak_idx] += smoothed_profile[idx + 1]
                idx += 1
            else:
                is_desc = False
        idx = peak_idx
        is_desc = True
        while is_desc:
            if idx > 0 and smoothed_profile[idx] >= smoothed_profile[idx - 1]and smoothed_profile[idx] != 0:
                peaks_wheight[peak_idx] += smoothed_profile[idx - 1]
                idx -= 1
            else:
                is_desc = False

    # Filter peaks on weight
    filtered_peaks = []
    for pos, count in peaks_wheight.items():
        if count >= max_noise*100:
            filtered_peaks.append(pos)
    
    # Return peaks
    return [idx for idx in filtered_peaks]


rows = []
titles = ["sample", "locus", "method", "expected", "predicted", "prediction_status"]
for curr_model in models_param:
    for spl in msi_eval:
        diamic_name = spl.name.replace("_L001", "").split("_")[0]
        if diamic_name in status_by_spl:
            if spl.loci[curr_model["id"]].results[method_name].getNbFrag() * 2 >= min_nb_reads:
                series = spl.loci[curr_model["id"]].results[method_name].getDensePrct(50, 200)
                smoothed_series = gaussian_filter1d(series, smooth_sigma)
                peaks = getPeaks(smoothed_series, 0.05)
                expected_status = status_by_spl[diamic_name][curr_model["name"]]
                predicted_status = "MSI" if len(peaks) > 1 else "MSS"
                rows.append([
                    spl.name.replace("_L001", ""),
                    curr_model["name"],
                    "smoothedPeaks",
                    expected_status,
                    predicted_status,
                    (expected_status == predicted_status)
                ])
                if expected_status != predicted_status:
                    print(
                        spl.name,
                        curr_model["name"],
                        getPeaks(smoothed_series),
                        find_peaks_cwt(series, np.arange(1,10), noise_perc=10),
                        status_by_spl[diamic_name][curr_model["name"]]
                    )
                    plt.plot(series, color='black')
                    plt.plot(smoothed_series, color='green')
                    plt.xlim(60, 110)
                    plt.show()
noML_df = pd.DataFrame.from_records(rows, columns=titles)

In [ ]:
# Plot evaluation
plt_by_locus = sns.factorplot(x="prediction_status", hue="method", col="locus", data=noML_df, order=[True, False, "NA"], kind="count")
for ax in plt_by_locus.axes.flat:
    locus_name = ax.get_title().split(" ")[-1]
    total_count = len(noML_df[(noML_df.locus == locus_name) & (noML_df.method == "smoothedPeaks")])
    for item in ax.patches:
        height = 0 if np.isnan(item.get_height()) else item.get_height()
        ax.annotate(
            '{:.1%}'.format(item.get_height()/total_count),
            (
                item.get_x() + item.get_width() / 2.,
                item.get_height()
            ),
            ha='center', va='center', fontsize=11, color='gray', rotation=90, xytext=(0, 20),
            textcoords='offset points')
plt.subplots_adjust(top=0.8)
plt.gcf().suptitle("Locus classification")
plt.show()